In [50]:
import csv
import json
import os
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define file paths using relative paths
base_theme_path = os.path.join('base-theme', 'base-theme.json')
variables_dark_path = os.path.join('input', 'variables_dark.csv')
variables_light_path = os.path.join('input', 'variables_light.csv')
themes_folder = 'themes'

# Ensure output folder exists
os.makedirs(themes_folder, exist_ok=True)

# Load the base theme file with utf-8-sig encoding to handle BOM
try:
    with open(base_theme_path, 'r', encoding='utf-8-sig') as file:
        base_theme = json.load(file)
    logging.info("Loaded base theme file.")
except Exception as e:
    logging.error(f"Failed to load base theme file: {e}")
    raise

# Function to set value in nested dictionary using dot notation and handling list indices
def set_nested_value(data, path, value):
    keys = re.split(r'\.|\[|\]\.?', path)
    keys = [key for key in keys if key]  # Remove empty strings
    for key in keys[:-1]:
        if key.isdigit():
            key = int(key)
            if isinstance(data, list):
                data = data[key]
            else:
                data = data.setdefault(key, [])
        else:
            data = data.setdefault(key, {})
    last_key = keys[-1]
    if last_key.isdigit():
        last_key = int(last_key)
        if isinstance(data, list):
            data[last_key] = value
        else:
            data[last_key] = value
    else:
        data[last_key] = value

# Function to load variables from a CSV file
def load_variables(variables_path):
    variables = {}
    try:
        with open(variables_path, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                variables[row['Variable Name']] = row['Hex']
        logging.info(f"Loaded variables from {variables_path}.")
    except Exception as e:
        logging.error(f"Failed to load variables from {variables_path}: {e}")
        raise
    return variables

# Function to load paths from a CSV file
def load_paths(paths_path):
    paths = []
    try:
        with open(paths_path, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                paths.append({
                    'Power BI Name': row['Power BI Name'],
                    'Variable': row['Variable'],
                    'Path': row['Path']
                })
        logging.info(f"Loaded paths from {paths_path}.")
    except Exception as e:
        logging.error(f"Failed to load paths from {paths_path}: {e}")
        raise
    return paths

# Function to generate theme files
def generate_theme(variables, paths, output_path):
    # Copy the base theme
    theme = json.loads(json.dumps(base_theme))

    # Update the theme with new colors
    for path_info in paths:
        hex_value = variables.get(path_info['Variable'])
        if hex_value:
            set_nested_value(theme, path_info['Path'], hex_value)

    # Save the new theme file in the output folder
    try:
        with open(output_path, 'w') as file:
            json.dump(theme, file, indent=4)
        logging.info(f"Generated theme file at {output_path}.")
    except Exception as e:
        logging.error(f"Failed to write theme file to {output_path}: {e}")
        raise

# Process each theme folder
for theme_folder in os.listdir(themes_folder):
    theme_path = os.path.join(themes_folder, theme_folder)
    if os.path.isdir(theme_path):
        paths_csv_path = os.path.join(theme_path, 'paths.csv')
        output_path = os.path.join(theme_path, f'{theme_folder}.json')

        # Determine whether to use dark or light variables
        if 'dark' in theme_folder.lower():
            variables_path = variables_dark_path
        elif 'light' in theme_folder.lower():
            variables_path = variables_light_path
        else:
            logging.warning(f"Skipping theme folder {theme_folder} due to unrecognized theme type.")
            continue

        # Load variables and paths
        variables = load_variables(variables_path)
        paths = load_paths(paths_csv_path)

        # Generate the theme
        generate_theme(variables, paths, output_path)

logging.info("Theme files generation completed.")

2024-05-23 15:55:54,702 - INFO - Loaded base theme file.
2024-05-23 15:55:54,703 - INFO - Loaded variables from input/variables_dark.csv.
2024-05-23 15:55:54,704 - INFO - Loaded paths from themes/dark-alternative/paths.csv.
2024-05-23 15:55:54,707 - INFO - Generated theme file at themes/dark-alternative/dark-alternative.json.
2024-05-23 15:55:54,708 - INFO - Loaded variables from input/variables_light.csv.
2024-05-23 15:55:54,708 - INFO - Loaded paths from themes/light-alternative/paths.csv.
2024-05-23 15:55:54,711 - INFO - Generated theme file at themes/light-alternative/light-alternative.json.
2024-05-23 15:55:54,711 - INFO - Loaded variables from input/variables_light.csv.
2024-05-23 15:55:54,712 - INFO - Loaded paths from themes/light/paths.csv.
2024-05-23 15:55:54,715 - INFO - Generated theme file at themes/light/light.json.
2024-05-23 15:55:54,716 - INFO - Loaded variables from input/variables_dark.csv.
2024-05-23 15:55:54,716 - INFO - Loaded paths from themes/dark/paths.csv.
202